This notebook is about training the vanilla CNN on MNIST

In [1]:
require 'torch'
require 'nn'
require 'nnx'
require 'optim'
require 'image'
require 'dataset-mnist' --This is a file which simply donwloads the data TODO: modify it for artificial one
require 'pl'
require 'paths'

We will fix seed for debugging

In [2]:
torch.manualSeed(1234)

We will set the number of threads, this is mostly required for CPU debugging. I have no GPUs on my laptop. We will also set the type to float since we are using SGD during training

In [15]:
torch.setnumthreads(4)
torch.setdefaulttensortype('torch.FloatTensor')
opt = {}
opt.plot = true
opt.batchSize = 10

Here, we define the basic architecture

In [4]:
classes = {'1','2','3','4','5','6','7','8','9','10'}
geometry = {32,32}
model = nn.Sequential()
-- stage 1 : mean suppresion -> filter bank -> squashing -> max pooling
model:add(nn.SpatialConvolutionMM(1, 32, 5, 5)) -- now it is 28x28x32
model:add(nn.Tanh())
model:add(nn.SpatialMaxPooling(3, 3, 3, 3)) -- now it is 8x8x32

-- stage 2 : mean suppresion -> filter bank -> squashing -> max pooling
model:add(nn.SpatialConvolutionMM(32, 64, 5, 5)) -- now it is 4x4x64
model:add(nn.Tanh())
model:add(nn.SpatialMaxPooling(2, 2, 2, 2)) -- now it is 2x2x64
-- stage 3 : standard 2-layer MLP:
model:add(nn.Reshape(64*2*2))
model:add(nn.Linear(64*2*2, 200))
model:add(nn.Tanh())
model:add(nn.Linear(200, #classes))
print(tostring(model))

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> output]
  (1): nn.SpatialConvolutionMM(1 -> 32, 5x5)
  (2): nn.Tanh
  (3): nn.SpatialMaxPooling(3,3,3,3)
  (4): nn.SpatialConvolutionMM(32 -> 64, 5x5)
  (5): nn.Tanh
  (6): nn.SpatialMaxPooling(2,2,2,2)
  (7): nn.Reshape(256)
  (8): nn.Linear(256 -> 200)
  (9): nn.Tanh
  (10): nn.Linear(200 -> 10)
}	


Add the loss function at the end

In [5]:
parameters,gradParameters = model:getParameters()
model:add(nn.LogSoftMax())
criterion = nn.ClassNLLCriterion()

We get the dataset

In [6]:
use_full = false
if use_full then
   nbTrainingPatches = 60000
   nbTestingPatches = 10000
else
   nbTrainingPatches = 2000
   nbTestingPatches = 1000
   print('<warning> only using 2000 samples to train quickly (modify use_fil to use 60000 samples)')
end

trainData = mnist.loadTrainSet(nbTrainingPatches, geometry)
trainData:normalizeGlobal(mean, std)

testData = mnist.loadTestSet(nbTestingPatches, geometry)
testData:normalizeGlobal(mean, std)

<warning> only using 2000 samples to train quickly (modify use_fil to use 60000 samples)	


<mnist> loading only 2000 examples	
<mnist> done	


<mnist> loading only 1000 examples	
<mnist> done	


In [7]:
_,t = trainData[1][2]:clone():max(1)
print(t)
t:squeeze()
print(t)

 6
[torch.LongTensor of size 1]

 6
[torch.LongTensor of size 1]



In [9]:
-- this matrix records the current confusion across classes
confusion = optim.ConfusionMatrix(classes)

-- log results to files
trainLogger = optim.Logger(paths.concat("logs", 'train.log'))
testLogger = optim.Logger(paths.concat("logs", 'test.log'))


In [17]:
-- training function
function train(dataset)
   -- epoch tracker
   epoch = epoch or 1

   -- local vars
   local time = sys.clock()

   -- do one epoch
   print('<trainer> on training set:')
   print("<trainer> online epoch # " .. epoch .. ' [batchSize = ' .. 10 .. ']')
   for t = 1,dataset:size(),opt.batchSize do
      -- create mini batch
      local inputs = torch.Tensor(opt.batchSize,1,geometry[1],geometry[2])
      local targets = torch.Tensor(opt.batchSize)
      local k = 1
      for i = t,math.min(t+opt.batchSize-1,dataset:size()) do
         -- load new sample
         local sample = dataset[i]
         local input = sample[1]:clone()
         local _,target = sample[2]:clone():max(1)
         target = target:squeeze()
         inputs[k] = input
         targets[k] = target
         k = k + 1
      end

      -- create closure to evaluate f(X) and df/dX
      local feval = function(x)
         -- just in case:
         collectgarbage()

         -- get new parameters
         if x ~= parameters then
            parameters:copy(x)
         end

         -- reset gradients
         gradParameters:zero()

         -- evaluate function for complete mini batch
         local outputs = model:forward(inputs)
         local f = criterion:forward(outputs, targets)

         -- estimate df/dW
         local df_do = criterion:backward(outputs, targets)
         model:backward(inputs, df_do)

         -- update confusion
         for i = 1,opt.batchSize do
            confusion:add(outputs[i], targets[i])
         end

         -- return f and df/dX
         return f,gradParameters
      end

    
      -- Perform SGD step:
      sgdState = sgdState or {
          learningRate = 0.05,
          momentum = 0,
          learningRateDecay = 5e-7
      }
      optim.sgd(feval, parameters, sgdState)
      
      -- disp progress
      xlua.progress(t, dataset:size())
   end
   
   -- time taken
   time = sys.clock() - time
   time = time / dataset:size()
   print("<trainer> time to learn 1 sample = " .. (time*1000) .. 'ms')

   -- print confusion matrix
   print(confusion)
   trainLogger:add{['% mean class accuracy (train set)'] = confusion.totalValid * 100}
   confusion:zero()

   -- save/log current net
   local filename = paths.concat("logs", 'mnist.net')
   os.execute('mkdir -p ' .. sys.dirname(filename))
   if paths.filep(filename) then
      os.execute('mv ' .. filename .. ' ' .. filename .. '.old')
   end
   print('<trainer> saving network to '..filename)
   -- torch.save(filename, model)

   -- next epoch
   epoch = epoch + 1
end

In [14]:
-- test function
function test(dataset)
   -- local vars
   local time = sys.clock()

   -- test over given dataset
   print('<trainer> on testing Set:')
   for t = 1,dataset:size(),opt.batchSize do
      -- disp progress
      xlua.progress(t, dataset:size())

      -- create mini batch
      local inputs = torch.Tensor(opt.batchSize,1,geometry[1],geometry[2])
      local targets = torch.Tensor(opt.batchSize)
      local k = 1
      for i = t,math.min(t+opt.batchSize-1,dataset:size()) do
         -- load new sample
         local sample = dataset[i]
         local input = sample[1]:clone()
         local _,target = sample[2]:clone():max(1)
         target = target:squeeze()
         inputs[k] = input
         targets[k] = target
         k = k + 1
      end

      -- test samples
      local preds = model:forward(inputs)

      -- confusion:
      for i = 1,opt.batchSize do
         confusion:add(preds[i], targets[i])
      end
   end

   -- timing
   time = sys.clock() - time
   time = time / dataset:size()
   print("<trainer> time to test 1 sample = " .. (time*1000) .. 'ms')

   -- print confusion matrix
   print(confusion)
   testLogger:add{['% mean class accuracy (test set)'] = confusion.totalValid * 100}
   confusion:zero()
end


In [ ]:
-- and train!
--
while true do
   -- train/test
   train(trainData)
   test(testData)

   -- plot errors
   if opt.plot then
      trainLogger:style{['% mean class accuracy (train set)'] = '-'}
      testLogger:style{['% mean class accuracy (test set)'] = '-'}
      trainLogger:plot()
      testLogger:plot()
   end
end


<trainer> on training set:	
<trainer> online epoch # 1 [batchSize = 10]	


Progress: 1691 / 2000	


<trainer> time to learn 1 sample = 3.4450534582138ms	


ConfusionMatrix:
[[     185       0       0       0       0       1       2       1       2       0]   96.859% 	[class: 1]
 [       0     213       1       3       1       0       0       2       0       0]   96.818% 	[class: 2]
 [       1       2     176       5       4       1       0       4       5       0]   88.889% 	[class: 3]
 [       0       0       2     176       0       6       0       3       2       2]   92.147% 	[class: 4]
 [       0       2       1       0     195       0       7       2       0       7]   91.121% 	[class: 5]
 [       0       1       2       3       4     165       3       0       1       1]   91.667% 	[class: 6]
 [       2       1       2       0       0       2     191       0       2       0]   95.500% 	[class: 7]
 [       0       1       6       0       4       0       0     202       0      11]   90.179% 	[class: 8]
 [       0       1       3       1       1       5       2       1     154       4]   89.535% 	[class: 9]
 [       3       0       1   

      1 : 1
      2 : 2
      3 : 3
      4 : 4
      

...

<trainer> time to test 1 sample = 1.8113820552826ms	


ConfusionMatrix:
[[      84       0       0       0       0       1       0       0       0       0]   98.824% 	[class: 1]
 [       0     126       0       0       0       0       0       0       0       0]   100.000% 	[class: 2]
 [       2       0     110       0       0       0       2       2       0       0]   94.828% 	[class: 3]
 [       0       0       1     102       0       4       0       0       0       0]   95.327% 	[class: 4]
 [       0       1       0       0     102       0       1       0       0       6]   92.727% 	[class: 5]
 [       0       0       1       1       0      84       1       0       0       0]   96.552% 	[class: 6]
 [       2       0       0       0       1       0      84       0       0       0]   96.552% 	[class: 7]
 [       0       0       2       1       0       0       0      93       0       3]   93.939% 	[class: 8]
 [       1       0       2       2       2       1       0       0      80       1]   89.888% 	[class: 9]
 [       0       0       0  

<trainer> on training set:	
<trainer> online epoch # 28 [batchSize = 10]	


Progress: 111 / 2000	


Progress: 631 / 2000	
